In [1]:
import ast
import pandas as pd
# Cargar el archivo CSV
file_path = "C:\\Users\\yanin\\Downloads\\movies_dataset.csv"
df = pd.read_csv(file_path)


# Convertir la columna 'popularity' a numérico
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

# Eliminar filas con valores nulos en 'popularity'
df_clean = df.dropna(subset=['popularity']).copy()

# Estratificar por cuartiles de popularidad
df_clean['popularity_bin'] = pd.qcut(df_clean['popularity'], q=4, duplicates='drop')

# Tomar una muestra estratificada de 1000 filas, evitando errores con pocos datos
sample_df = df_clean.groupby('popularity_bin', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 1000 // df_clean['popularity_bin'].nunique()), random_state=42)
).copy()

# Eliminar la columna auxiliar
sample_df = sample_df.drop(columns=['popularity_bin'])

# Convertir la columna 'belongs_to_collection' a diccionarios
sample_df["belongs_to_collection"] = sample_df["belongs_to_collection"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and pd.notna(x) else None
)

# Desanidar valores en un nuevo DataFrame
df_collection = sample_df["belongs_to_collection"].apply(
    lambda x: {"idCollection": x.get("id"), "nameCollection": x.get("name")} if isinstance(x, dict) else None
).apply(pd.Series)

# Convertir 'idCollection' a Int64 (soporta valores nulos)
df_collection["idCollection"] = df_collection["idCollection"].astype("Int64")

# Verificar valores nulos en release_date antes y después de la conversión
null_dates = df["release_date"].isnull().sum()
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
null_dates_after = df["release_date"].isnull().sum()

# Extraer el año de la fecha de estreno
df["release_year"] = df["release_date"].dt.year

# Eliminar filas donde 'release_date' sigue siendo nulo
df = df.dropna(subset=['release_date'])

# Verificar el cambio
df[['release_date', 'release_year']].head()

# Convertir 'revenue' y 'budget' a valores numéricos, reemplazando errores con 0
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce").fillna(0)
df["budget"] = pd.to_numeric(df["budget"], errors="coerce").fillna(0)

# Calcular el retorno de inversión (ROI)
df["return"] = df.apply(lambda x: x["revenue"] / x["budget"] if x["budget"] > 0 else 0, axis=1)

# Mostrar las primeras filas con la nueva columna
df[["revenue", "budget", "return"]].head()

# Eliminar las columnas no utilizadas
columns_to_drop = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"]
df = df.drop(columns=columns_to_drop, errors="ignore")

# Verificar que las columnas fueron eliminadas
df.head()

# Guardar el DataFrame modificado en un nuevo archivo CSV
output_file_path = "C:\\Users\\yanin\\Downloads\\movies_dataset_modified.csv"
df.to_csv(output_file_path, index=False)

print(f"Archivo guardado en: {output_file_path}")


C:\Users\yanin\AppData\Local\Temp\ipykernel_2512\2858094301.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\yanin\AppData\Local\Temp\ipykernel_2512\2858094301.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sample_df = df_clean.groupby('popularity_bin', group_keys=False).apply(
C:\Users\yanin\AppData\Local\Temp\ipykernel_2512\2858094301.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = df_cle

Archivo guardado en: C:\Users\yanin\Downloads\movies_dataset_modified.csv
